In [5]:
import requests # Импортируем библиотеку pandas и другие
from requests.exceptions import ConnectionError
from time import sleep
import json
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from pprint import pprint
from openpyxl import load_workbook
from openpyxl import Workbook
from pprint import pprint

In [6]:
# Библиотеки для работы с Google Sheets API
import httplib2
import argparse
from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage
import gspread # Библиотека для записи в Google sheets
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
# Авторизация в Google Sheets API
SCOPES = 'https://www.googleapis.com/auth/spreadsheets' # Разрешения на просмотр и редактирование
CLIENT_SECRET_FILE = 'promoshift-sheets-client-secret-grants.json' # Название файла с идентификаторами
APPLICATION_NAME = 'Google Sheets API Report' # Название приложения для нашего онлайн-отчета (можно использовать любое)

credential_path = 'promoshift-sheets.googleapis.com-report.json'

store = Storage(credential_path)
credentials = store.get()
parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter, 
                                 parents=[tools.argparser])
flags = parser.parse_args([])

if not credentials or credentials.invalid:
    flow = client.flow_from_clientsecrets(r'/Users/egoshin/Downloads/promoshift-sheets-client-secret-grants.json', SCOPES)
    flow.user_agent = APPLICATION_NAME
    if flags:
        credentials = tools.run_flow(flow, store,flags)
    print('Storing credentials to ' + credential_path)

In [4]:
# Формируем объект 'service', с помощью которого мы будем читать и записывать данные Google Sheets

http = credentials.authorize(httplib2.Http())
discoveryUrl = ('https://sheets.googleapis.com/$discovery/rest?version=v4')
service = discovery.build('sheets', 'v4', http = http, discoveryServiceUrl=discoveryUrl)

In [32]:
with open(r'/Users/egoshin/Downloads/ya_token.json') as f:
    data = json.load(f)
    f.close()
token = data

with open(r'/Users/egoshin/Downloads/ya_campaigns.json') as f:
    data = json.load(f)
    f.close()
campaigns = data
aggStats = {}

spreadsheet_id = '10jZ_61x7EWDGWwJBWFfrOjxHiDBxGGpatP44ubMhMpc'

In [6]:
# -*- coding: utf-8 -*-
import requests
from requests.exceptions import ConnectionError
from time import sleep
import json

# Метод для корректной обработки строк в кодировке UTF-8 как в Python 3, так и в Python 2
import sys

if sys.version_info < (3,):
    def u(x):
        try:
            return x.encode('utf8')
        except UnicodeDecodeError:
            return x
else:
    def u(x):
        if type(x) == type(b''):
            return x.decode('utf8')
        else:
            return x

# --- Входные данные ---
# Адрес сервиса Reports для отправки JSON-запросов (регистрозависимый)
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

# OAuth-токен пользователя, от имени которого будут выполняться запросы
token = token

def report(login):

    # Логин клиента рекламного агентства
    # Обязательный параметр, если запросы выполняются от имени рекламного агентства
    clientLogin = login

    # --- Подготовка запроса ---
    # Создание HTTP-заголовков запроса
    headers = {
               'Authorization': 'Bearer ' + token, # OAuth-токен. Использование слова Bearer обязательно
               'Client-Login': clientLogin, # Логин клиента рекламного агентства
               'Accept-Language': 'ru', # Язык ответных сообщений
               'processingMode': 'auto', # Режим формирования отчета
               'returnMoneyInMicros': 'false', # Формат денежных значений в отчете
               # 'skipReportHeader': 'true', # Не выводить в отчете строку с названием отчета и диапазоном дат
               # 'skipColumnHeader': 'true', # Не выводить в отчете строку с названиями полей
               # 'skipReportSummary': 'true' # Не выводить в отчете строку с количеством строк статистики
               }

    # Создание тела запроса
    body = {
        'params': {
            'SelectionCriteria': {
                # 'DateFrom': '2020-03-01', # Используется только при наличии 'CUSTOM_DATE'. В 'CUSTOM_REPORT' почему-то не работает
                # 'DateTo': '2020-03-29', 
                # 'Filter': [{'Field': 'CampaignId', 'Operator': 'IN', 'Values': campaigns}], 
          },
            'FieldNames': [
                'CampaignId', 
                'CampaignName', 
                'Clicks', 
                'Cost', 
            ],
            'ReportName': 'MonthlyPerformanceReport',
            'ReportType': 'CUSTOM_REPORT', # 'CUSTOM_REPORT', 'CAMPAIGN_PERFORMANCE_REPORT'
            'DateRangeType': 'LAST_MONTH', # 'THIS_MONTH', 'LAST_MONTH' or 'CUSTOM_DATE'
            'Format': 'TSV',
            'IncludeVAT': 'NO',
            'IncludeDiscount': 'NO', 
        }
    }

    # Кодирование тела запроса в JSON
    body = json.dumps(body, indent=4)

    # --- Запуск цикла для выполнения запросов ---
    # Если получен HTTP-код 200, то выводится содержание отчета
    # Если получен HTTP-код 201 или 202, выполняются повторные запросы

    while True:
        try:
            req = requests.post(ReportsURL, body, headers=headers)
            # req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
            if req.status_code == 400:
                print('Параметры запроса указаны неверно или достигнут лимит отчетов в очереди')
                print('RequestId: {}'.format(req.headers.get('RequestId', False)))
                print('JSON-код запроса: {}'.format(u(body)))
                print('JSON-код ответа сервера: \n{}'.format(u(req.json())))
                break
            elif req.status_code == 200:
                print('Отчет загружен успешно')
                # print('RequestId: {}'.format(req.headers.get('RequestId', False)))
                # print('Содержание отчета: \n{}'.format(u(req.text)))
                format((req.text))
                return req.text
                break
            elif req.status_code == 201:
                print('Отчет успешно поставлен в очередь в режиме офлайн')
                retryIn = int(req.headers.get('retryIn', 60))
                print('Повторная отправка запроса через {} секунд'.format(retryIn))
                print('RequestId: {}'.format(req.headers.get('RequestId', False)))
                sleep(retryIn)
            elif req.status_code == 202:
                print('Отчет формируется в режиме офлайн')
                retryIn = int(req.headers.get('retryIn', 60))
                print('Повторная отправка запроса через {} секунд'.format(retryIn))
                print('RequestId:  {}'.format(req.headers.get('RequestId', False)))
                sleep(retryIn)
            elif req.status_code == 500:
                print('При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее')
                print('RequestId: {}'.format(req.headers.get('RequestId', False)))
                print('JSON-код ответа сервера: \n{}'.format(u(req.json())))
                break
            elif req.status_code == 502:
                print('Время формирования отчета превысило серверное ограничение.')
                print('Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.')
                print('JSON-код запроса: {}'.format(body))
                print('RequestId: {}'.format(req.headers.get('RequestId', False)))
                print('JSON-код ответа сервера: \n{}'.format(u(req.json())))
                break
            else:
                print('Произошла непредвиденная ошибка')
                print('RequestId:  {}'.format(req.headers.get('RequestId', False)))
                print('JSON-код запроса: {}'.format(body))
                print('JSON-код ответа сервера: \n{}'.format(u(req.json())))
                break

        # Обработка ошибки, если не удалось соединиться с сервером API Директа
        except ConnectionError:
            # В данном случае мы рекомендуем повторить запрос позднее
            print('Произошла ошибка соединения с сервером API')
            # Принудительный выход из цикла
            break

        # Если возникла какая-либо другая ошибка
        except:
            # В данном случае мы рекомендуем проанализировать действия приложения
            print('Произошла непредвиденная ошибка')
            # Принудительный выход из цикла
            break

In [7]:
# -------- Создание Excell-файла на компьютере, куда будут записаны данные из выгрузки --------

path = r'/Users/egoshin/Downloads/MonthlyPerformanceReport.xlsx'
writer = pd.ExcelWriter(path, engine='openpyxl')
book = Workbook('MonthlyPerformanceReport.xlsx')
book.save(path)
book = load_workbook(path)
for account in campaigns:
    book['Sheet'].append([account])
writer.book = book

# -------- Данные для работы с библиотекой 'gspread' --------

scope = ['https://spreadsheets.google.com/feeds', 
         'https://www.googleapis.com/auth/drive']

# Даем ссылку на второй - скаченный из 'Google developers console' файл с сервис-ключами для Google sheets
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    r'/Users/egoshin/Downloads/promoshift-google-sheets-api-key.json', scope)
gc = gspread.authorize(credentials) # Авторизация библиотеки 'gspread'
sh = gc.open('MonthlyPerformanceReport') # Создание объекта файла в Google sheets

# -------- Проверка на наличие всех необходимых листов в Google sheets --------

names = [] # Пустой список, куда добавляем названия листов из полученного ниже списка
# Получение списка уже имеющихся листов с логинами клиентов
lists = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()

# Цикл получения названий листов из Google sheets
i = 0
for name in lists['sheets']:
    names.append(lists['sheets'][i]['properties']['title'])
    i += 1

# Если логина клиента нет в списке листов - создаем в Spreadsheet новый лист с логином клиента
for login in campaigns:
    if login not in names:
        sh.add_worksheet(title=login, rows='100', cols='20')   

# -------- Создание датафрейма на основе данных выгрузки по API --------        

for login in campaigns:
    sheet = report(login)
    df = pd.DataFrame(sheet.rsplit(sep='\n')) # Преобразование данных в датафрейм и разделение по '\n'
    df = [i.split(sep='\t') for i in df[0]] # Разделение по '\t'
    date = df[0] # Название репорта с датами выборки
    df = pd.DataFrame(df)
    df = df.drop(df.index[[0,1]]).reset_index(drop=True).dropna()
    df.columns = ['CampaignId', 'CampaignName', 'Clicks', 'Cost']
    df['Clicks'] = df['Clicks'].astype(float)
    df['Cost'] = df['Cost'].astype(float)
    
    for city in campaigns[login]:
        aggByCity = (city, df[df['CampaignId'].isin(campaigns[login][city])].agg({'Clicks':'sum', 
                                                                 'Cost':'sum'}))
        if city not in aggStats.keys():
                aggStats[city] = df[df['CampaignId'].isin(campaigns[login][city])].agg({'Clicks':'sum', 
                                                                 'Cost':'sum'})
    
    aggStatsDf = pd.DataFrame(aggStats).transpose()
    aggStatsDf.fillna(0)
    aggStatsDf.index.name = 'GroupBy' # Переименуем индекс датафрейма - понадобится для имопрта в Google sheets
    
    # -------- На основе полученных столбцов - вычислим и добавим новые --------
    
    aggStatsDf['CostWithNDS'] = [cost*1.2 for cost in aggStatsDf['Cost']]
    aggStatsDf['CostWithNDS'] = aggStatsDf['CostWithNDS'].round(2)
    aggStatsDf['AvgCPC'] = (aggStatsDf['CostWithNDS']/aggStatsDf['Clicks']).round(2)
    
    # -------- Импорт в Google sheets --------
    
    to_list = aggStatsDf.fillna(0).reset_index().values.tolist() # Преобразуем датафрейм  лист для импорта в Google sheets
    
    headlines = {'values': [['GroupBy', 'Clicks', 'Cost', 'CostWithNDS', 'AvgCPC']]} # Добавим заголовки
    service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=str(login), 
                                               valueInputOption='USER_ENTERED', 
                                               body=headlines).execute()
    
    body = {'values': to_list} # Добавим основные данные
    service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=str(login)+'!A2', 
                                               valueInputOption='USER_ENTERED', 
                                               body=body).execute()
    
    print('Отчет по {} записан в Google sheets: MonthlyPerformanceReport'.format(login))
    
    # -------- Запись в Excell на компьютере --------
    
    aggStatsDf.to_excel(writer, index=True, sheet_name=login)
    writer.save()
    print('Отчет по {} создан успешно'.format(login))
    aggStats.clear()

# -------- Указываем список аккаунтов для которых будут сделаны репорты и диапазон дат --------        

# Чистим лист с датой и списком аккаунтов
service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, 
                                      range='accounts', 
                                      body={}).execute()

# Диапазон дат
service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range='accounts', 
                                           valueInputOption='USER_ENTERED', 
                                           body={'values': [date]}).execute()

# Список аккаунтов
for account in campaigns:
    body = {'values': [[account]]}
    service.spreadsheets().values().append(spreadsheetId=spreadsheet_id, range='accounts!A3', 
                                           valueInputOption='USER_ENTERED', 
                                           body=body).execute()
    
print('----------------------\nОтчет полностью создан: https://docs.google.com/spreadsheets/d/10jZ_61x7EWDGWwJBWFfrOjxHiDBxGGpatP44ubMhMpc/edit#gid=1962875708\n----------------------\n')

Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 2514829395917326616
Отчет загружен успешно
Отчет по nakolesahmira записан в Google sheets: MonthlyPerformanceReport
Отчет по nakolesahmira создан успешно
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 2514833040309367796
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  2514834807867984824
Отчет загружен успешно
Отчет по buyusa-promoshift записан в Google sheets: MonthlyPerformanceReport
Отчет по buyusa-promoshift создан успешно
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 2514848165077171800
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  2514849913885849505
Отчет загружен успешно
Отчет по nakolesahmira-farkopy записан в Google sheets: MonthlyPerformanceReport
Отчет по nakolesahmira-farkopy со

Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 2515045153856141097
Отчет загружен успешно
Отчет по playstand-promoshift записан в Google sheets: MonthlyPerformanceReport
Отчет по playstand-promoshift создан успешно
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 2515048747485726760
Отчет загружен успешно
Отчет по express-42-promoshift записан в Google sheets: MonthlyPerformanceReport
Отчет по express-42-promoshift создан успешно
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 секунд
RequestId: 2515052328499385558
Отчет формируется в режиме офлайн
Повторная отправка запроса через 10 секунд
RequestId:  2515054229588591067
Отчет загружен успешно
Отчет по kama-tyres-ekb записан в Google sheets: MonthlyPerformanceReport
Отчет по kama-tyres-ekb создан успешно
Отчет успешно поставлен в очередь в режиме офлайн
Повторная отправка запроса через 1 сек